In [146]:
import pandas as pd
import time
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from IPython.display import display, HTML

In [145]:
def get_ufcstats_links():
    url = "http://ufcstats.com/statistics/events/completed"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    event_urls = []
    fight_details_urls = []

    for url in soup.select(".b-statistics__table-content a"):
        if url.has_attr("href"):
            event_urls.append(url["href"])
    
    for event_url in event_urls[:3]:
        response = requests.get(event_url)
        soup = BeautifulSoup(response.text, "html.parser")

        for i in soup.select("tr.b-fight-details__table-row"):
            if i.has_attr("data-link"):
                fight_details_urls.append(i["data-link"])

    return fight_details_urls
    

In [144]:
def extracting_details(match_url):
    response = requests.get(match_url)
    soup = BeautifulSoup(response.text, "html.parser")

    fighters = [name.text.strip() for name in soup.select(".b-fight-details__person-name")]
    result = [status.text.strip() for status in soup.select(".b-fight-details__person-status")]
    method = soup.select_one(".b-fight-details__text-item_first:-soup-contains('Method')")
    round = soup.select_one(".b-fight-details__text-item:-soup-contains('Round')")
    time = soup.select_one(".b-fight-details__text-item:-soup-contains('Time')")
    time_format = soup.select_one(".b-fight-details__text-item:-soup-contains('Time format')")
    referee = soup.select_one(".b-fight-details__text-item:-soup-contains('Referee')")
    details = [detail.text.replace("\n", "") for detail in soup.select(".b-fight-details__text")]

    fighter1_details = {
        "match_url": match_url,
        "fighter": fighters[0] if len(fighters) > 1 else "null",
        "fighting_against": fighters[1] if len(fighters) > 1 else "null",
        "result": result[0] if len(result) > 1 else "null",
        "method": method.text.strip().split("\n")[-1] if method else "null",
        "round": round.text.strip().split("\n")[-1] if round else "null",
        "time": time.text.strip().split("\n")[-1] if time else "null",
        "time_format": time_format.text.strip().split("\n")[-1] if time_format else "null",
        "referee": referee.text.strip().split("\n")[-1] if referee else "null",
        "details": details[1] if len(details) > 1 else "null",
    }

    fighter2_details = {
        "match_url": match_url,
        "fighter": fighters[1] if len(fighters) > 1 else "null",
        "fighting_against": fighters[0] if len(fighters) > 1 else "null",
        "result": result[1] if len(result) > 1 else "null",
        "method": method.text.strip().split("\n")[-1] if method else "null",
        "round": round.text.strip().split("\n")[-1] if round else "null",
        "time": time.text.strip().split("\n")[-1] if time else "null",
        "time_format": time_format.text.strip().split("\n")[-1] if time_format else "null",
        "referee": referee.text.strip().split("\n")[-1] if referee else "null",
        "details": details[1] if len(details) > 1 else "null",
    }

    return fighter1_details, fighter2_details

In [143]:
def main():
    fight_details_list = get_ufcstats_links()
    list1, list2 = [], []
    fights = []


    for i in fight_details_list[30:32]:
        list1, list2 = extracting_details(i)
        fights.append(list1)
        fights.append(list2)
        time.sleep(3)

    df_fights = pd.DataFrame(fights)
    display(HTML(df_fights.to_html()))

    print('Done.')

In [ ]:
if __name__ == '__main__':
    main()